# Real Time Face Emotion Detection using Convolutional Neural Network (CNN)

## Introduction
Seiring berkembangnya teknologi banyak penemuan diciptakan dengan menggunakan berbagai metode machine learning maupun deep learning. Salah satunya yang paling banyak digunakan adalah facial detection atau sistem pengenalan wajah. Dengan menggunakan sistem pengenalan wajah, kita dapat melakukan indentifikasi terhadap identitas bahkan emosi dari wajah yang bersangkutan. 
Kita dapat melakukan sistem pengenalan wajah secara real time dengan menggunakan web camera , cctv, ataupun melalui foto.
Umumnya sistem pengenalan wajah ini dibangun menggunakan metode deep learning atau openCV.
Pada notebook ini, kita akan belajar bagaimana cara mengidentifikasi emosi seseorang secara real time dengan web camera menggunakan metode deep leaning CNN (Convolutional Neural Network).

## Convolutional Neural Network (CNN)
Dalam Neural Network (NN), CNN adalah model utama yang dapat digunakan untuk melakukan image recognition dan image classification. Secara teknis, CNN dipecah terlebih dahulu menjadi data train dan data test. Setiap input image yang masuk akan melalui convolutional layers kernel. Untuk detail prosesnya, dapat dilihat melalui gambar berikut ini :<br>
<img src="assets/cnn.JPG" width="700" />

Jika dilihat pada gambar diatas, dapat kita simpulkan bahwa CNN dibagi menjadi dua bagian besar, yaitu Feature Learning dan Classification. <br>

#### Feature Learning : 
Pada tahapan Feature Learning terjadi proses ekstrasi image menjadi features angka yang merepresentasikan image tersebut, umumnya angka-angka tersebut dikemas dalam bentuk vector. Feature learning terdiri dari beberapa bagian, yaitu :<br>

**1. Convolutional Layer :**<br>
Convolutional layer terdiri dari neuron tersusun yang membentuk sebuah filter, dimana filter tersebut akan digeser keseluruh bagian dari image. Setiap pergeseran akan dilakukan operasi perkalian (dot) antara input dan filter tersebut sehingga menghasilkan output yang disebut feature map. Berikut dibawah ini adalah ilustrasi proses convolutional layer :
![SegmentLocal](assets/cl.gif "segment")

**2. Pooling Layers :**<br>
Pooling layer biasanya terletak setelah convolutional layer. Pooling layer ini berfungsi untuk mengurangi overfitting dan mempercepat komputasi. Secara teknis, pooling layer akan mengurangi dimensi dari hasil output (feature map) pada convolutional layer. Pooling yang biasa digunakan adalah MaxPooling dan AveragePooling. Pada notebook ini, kita akan menggunakan MaxPooling untuk proses reduksi dimensinya.<br>

**3. ReLU (Rectified Linear Unit) :**<br>
ReLU pada proses ini berfungsi melakukan aktivasi element untuk mengurangi vanishing gradient. ReLU merupakan operasi non-linear yang memiliki fungsi ƒ(x) = max(0,x).

#### Classification :
Tahapan setelah ekstrasi fitur adalah proses klasifikasi (classification). Proses classification terdiri dari : <br>

**1. Flatten :**<br>
Flatten digunakan untuk mengubah feature map kedalam bentuk vector yang kemudian akan digunakan sebagai input pada fully-connected layer.<br>

**2. Fully-connected layer :**<br>
<img src="assets/fc.JPG" width="600" />

Proses flatten akan menghasilkan vector yang akan dijadikan sebagai input, jika dilihat pada gambar diatas, input tersebut antara lain adalah x1,x2,x3,dst. Dengan fully-connected layer, kita akan mengkombinasikan beberapa input tersebut menjadi sebuah model yang kemudian akan dihitung bobot outputnya. <br>

**3. Softmax :**<br>
Fungsi softmax digunakan untuk menghitung probabilitas dari setiap kelas target. Probabilitas inilah yang digunakan untuk mengklasifikasikan image input.


## Demo Program

#### Informasi Dataset
Dataset yang digunakan adalah dataset `fer2013.csv` yang dapat diakses pada link berikut [ini](https://www.kaggle.com/deadskull7/fer2013).<br>
Dataset ini terdiri dari 2 kolom, yaitu `emotion` dan `pixel`, dimana pada kolom `emotion` terdapat nilai 0-6 yang bertipe categorical dan mewakili setiap emosi yang akan dideteksi.  (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral). Sedangkan kolom `pixel` memuat informasi nilai pixel dari masing-masing emosi.


In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('data_input/fer2013.csv')

width, height = 48, 48

datapoints = data['pixels'].tolist()

X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

y = pd.get_dummies(data['emotion'])

np.save('fdataX', X)
np.save('flabels', y)

#### Modelling CNN
Pada proses modelling ini, pooling yang digunakan adalah MaxPooling dengan jumlah iterasi epoch sebanyak 100 kali. 

In [ ]:
import sys, os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.models import model_from_json


num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

x = np.load('fdataX.npy')
y = np.load('flabels.npy')

x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

np.save('modXtest', X_test)
np.save('modytest', y_test)

#model CNN
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_valid), np.array(y_valid)),
          shuffle=True)

fer_json = model.to_json()
with open("model_fer_new.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("model_weight_new.h5")

Train on 29068 samples, validate on 3230 samples
Epoch 1/100
29068/29068 [==============================] - 34s 1ms/step - loss: 1.9968 - accuracy: 0.2155 - val_loss: 1.8087 - val_accuracy: 0.2594
Epoch 2/100
29068/29068 [==============================] - 33s 1ms/step - loss: 1.8073 - accuracy: 0.2596 - val_loss: 1.7414 - val_accuracy: 0.2994
Epoch 3/100
29068/29068 [==============================] - 33s 1ms/step - loss: 1.7221 - accuracy: 0.3077 - val_loss: 1.5887 - val_accuracy: 0.3505
Epoch 4/100
29068/29068 [==============================] - 33s 1ms/step - loss: 1.5990 - accuracy: 0.3674 - val_loss: 1.4473 - val_accuracy: 0.4331
Epoch 5/100
29068/29068 [==============================] - 33s 1ms/step - loss: 1.5137 - accuracy: 0.4134 - val_loss: 1.3953 - val_accuracy: 0.4498
Epoch 6/100
29068/29068 [==============================] - 33s 1ms/step - loss: 1.4412 - accuracy: 0.4446 - val_loss: 1.3586 - val_accuracy: 0.4734
Epoch 7/100
29068/29068 [==============================] - 33s 

Proses modelling diatas menghasilkan bobot dan model yang kemudian disimpan dalam ekstensi file json dan h5.
Jika kita melihat hasil pada proses modelling diatas, maka diperoleh informasi bahwa terjadi overfitting. Hal ini dapat dilihat dari nilai akurasi pada training set yang berkisar 90% lebih, sedangkan akurasi pada test set nya hanya berkisar 65% saja.<br>
Kita dapat melakukan improvement pada model untuk menaikkan nilai akurasinya dengan cara menambah jumlah iterasi epoch, mengurangi nilai `stride` agar informasi yang diperoleh lebih detail, atau menambah nilai dropout.

#### Deteksi emosi realtime dengan menggunakan webcam
Dibawah ini adalah kode program yang digunakan untuk melakukan deteksi wajah dan emosi secara realtime. Proses deteksi wajah menggunakan pre-train model Haar Cascade, sedangkan deteksi emosi menggunakan nilai bobot dari hasil modelling CNN diatas. 

In [1]:
from keras.preprocessing import image
import cv2
import numpy as np
import pandas as pd
from keras.models import model_from_json

model = model_from_json(open("Model/model_fer_new.json", "r").read())
model.load_weights('Model/model_weight_new.h5')

face_haar_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml')


cap=cv2.VideoCapture(0)

while True:
    ret,test_img=cap.read()
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)

    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)



    if cv2.waitKey(10) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows


Using TensorFlow backend.
C:\Users\litaimut\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\litaimut\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\litaimut\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\litaimut\anaconda3\lib\site-packages\tensorflow\python\framewo

Instructions for updating:
Colocations handled automatically by placer.


<function destroyAllWindows>

Ketika kode program tersebut diatas dijalankan, maka akan menampilkan output window dan secara otomatis webcam pada device laptop kita akan menyala. Berikut dibawah ini adalah 2 hasil deteksi emosi secara real time :

**1. Senang**
<img src="assets/happy.png" width="400" />

**2. Netral**
<img src="assets/neutral.JPG" width="400" />

Hasil deteksi emosi tersebut diatas masih memiliki akurasi yang tergolong rendah, sehingga hanya beberapa emosi saja yang dapat di deteksi dengan tepat.

## Conclusion
1. Terdapat overfitting pada saat melakukan modelling dataset `fer2013.csv` menggunakan algoritma CNN.
2. Nilai akurasi untuk data training sebesat 90.6%, sedangkan nilai akurasi untuk data test sebesar 65%.
3. Overfitting dapat diatasi dengan menambah jumlah iterasi epoch, mengurangi parameter `stride` agar informasi yang dihasilkan lebih detail, atau menambah parameter nilai dropout.